This notebook will explore the photon occupation of the ground-state of MgH+ in the ultrastrong coupling regime.  This analysis is inspired by Figure 2a in the paper 
["Virtual photons in the ground state of a dissipative system"](https://www.nature.com/articles/s41467-017-01504-5)
 by Simone De Liberato.  The data generated in this notebook corresponds to Figure 4 in McTague and Foley.


In [ ]:
from __future__ import print_function

"""
A script to run the cqed_rhf and cqed_cis method on MgH+ potential energy surface in a cc-pVDZ basis set,
reproducing data from Figure 3 by McTague and Foley.
"""

__authors__   = ["Jon McTague", "Jonathan Foley"]
__credits__   = ["Jon McTague", "Jonathan Foley"]

__copyright_amp__ = "(c) 2014-2018, The Psi4NumPy Developers"
__license__   = "BSD-3-Clause"
__date__      = "2021-01-15"

# ==> Import Psi4, NumPy, & SciPy <==
import psi4
import numpy as np
from helper_cqed_rhf import *
from helper_ciss_prism import *
from matplotlib import pyplot as plt
# Set Psi4 & NumPy Memory Options
psi4.set_memory('2 GB')
psi4.core.set_output_file('output.dat', False)

numpy_memory = 2


We will explore the photon occupation in the ground state under different cavity coupling
and dissipation rates.  In particular, we will consider cavity coupling strengths $g$ relative 
to the photon energy / transition energy as follows:

$$ g = N\frac{\omega}{2}, $$
where $N = 0.5, 1, 1.5, 2$.

We can estimate $g$ as 

$$ g = \sqrt{\frac{\omega}{2}}\lambda \mu $$, 

where we can rearrange for $\lambda$:

$$ \lambda = \frac{f}{\mu \sqrt{\frac{1}{2\omega}}} $$
where $f = 0., 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45,  0.5$.

Additionally, we will consider an imaginary part of the frequency $\gamma$ such 
that 
$$ \gamma = m \omega, $$
where $m = 0.0, 0.5, 1.0, 1.5, 2.0$.

In [ ]:
# template for z-matrix
molstr = """
Mg
H 1 2.2
symmetry c1
1 1
"""

# options dict
options_dict = {'basis': 'cc-pVDZ',
               'save_jk': True, 
               'scf_type': 'pk'}

# set psi4 options
psi4.set_options(options_dict)

# approximate maximum value of transition dipole moment in a.u.
max_tdm = 2.25

# fix real part of omega
om_r = 4.75 / 27.211

# approximate max value of g
#g_max = np.sqrt(om_r/2) * lam_z * max_tdm

# this is a list of fraction values that is used to compute the value
# of \lambda such that the g/\omega is a fraction with the following magnitudes
f_list = np.array([0., 0.05, 0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45,  0.5])

# this is a list of fraction values that is used to define the \gamma value
# relative to the real part of the frequency \omega
m_list = np.array([0, 0.5, 1.0, 1.5, 2.0])

# photon energy
om_r = 4.75 / 27.211

# define the different values of \lambda
lambda_list = f_list / (max_tdm * np.sqrt(1/(2 * om_r)))
print("lambda list")
print(lambda_list)


# get in terms of g
g_list = np.sqrt(om_r/2) * lambda_list * max_tdm
print("g list")
print(g_list)

# define the different values of \gamma
gamma_list = om_r * m_list
print("gamma list")
print(gamma_list)

relative_strength = np.zeros(len(f_list))
# array to hold the cs-cqed-cis energies
#cqed_cis_energy_array = np.zeros((5, n_lam),dtype=complex)

# array to hold the photon occupations of the ground state
photon_occupation_zo = np.zeros(len(f_list))
photon_occupation_fo = np.zeros(len(f_list))
photon_occupation_so = np.zeros(len(f_list))
photon_occupation = np.zeros(len(f_list))



# loop over the different bond-lengths, create different instances
# of MgH+ molecule and compute and store various quantities
n_state = 0
l_ctr = 0
mol = psi4.geometry(molstr)
e, wfn = psi4.energy("scf/cc-pVDZ", return_wfn=True, molecule=mol)
for l in lambda_list:
    lam = np.array([0, 0, l])
    om = om_r 
    cqed_dict = cs_cqed_cis(lam, om, molstr, options_dict)
    g_c = np.sqrt(om/2) * l * max_tdm
    relative_strength[l_ctr] = np.real( np.sqrt( np.conj(g_c) * g_c ))
    C = cqed_dict['C VECTOR']
    r1_offset = cqed_dict["R1 OFFSET"]
    H_DSE = cqed_dict["H CISS-DSE"]
    H_BLC = cqed_dict["H CISS-BLC"]
    po_l = 0.
    for i in range(r1_offset, len(C[:,0])):
        po_l += np.real(np.conj(C[i,n_state]) * C[i,n_state])
    photon_occupation_zo[l_ctr] = po_l
    
    fo = np.dot(C[:,n_state].T, H_BLC).dot(C[:,n_state])
    # matrix already contains the -1/sqrt(2\omega) facor
    photon_occupation_fo[l_ctr] = fo
    
    so = np.dot(C[:,n_state].T, H_DSE).dot(C[:,n_state])
    # this matrix does not contain the factor 1 / 2\omega term so we capture it here
    photon_occupation_so[l_ctr] = 1 / (2 * om) * so
    
    photon_occupation[l_ctr] = po_l + fo + 1 / (2 * om) * so
    
    l_ctr += 1

    

 

In [ ]:


#plt.plot(relative_strength/om_r, photon_occupation_zo, '-bo', label="0th order")
#plt.plot(relative_strength/om_r, photon_occupation_fo, '-go', label="1st order")
#plt.plot(relative_strength/om_r, photon_occupation_so, '-ro', label="2nd order")
plt.plot(lambda_list, photon_occupation, '-o', label="Total")

#plt.plot(relative_strength[1,:]/om_r, photon_occupation_L[1,:], 'y--')
#plt.plot(relative_strength[2,:]/om_r, photon_occupation_L[2,:], 'g--')
#plt.plot(relative_strength[3,:]/om_r, photon_occupation_L[3,:], 'r--')
#plt.plot(relative_strength[4,:]/om_r, photon_occupation_L[4,:], '-o')
#plt.xlim(0,0.65)
#plt.ylim(0,0.045)
plt.legend()
plt.show()

#for i in range(0, len(relative_strength[1,:])):
#    print(relative_strength[0,i]/om_r, photon_occupation_L[0,i],
#          relative_strength[1,i]/om_r, photon_occupation_L[1,i],
#          relative_strength[2,i]/om_r, photon_occupation_L[2,i],
#          relative_strength[3,i]/om_r, photon_occupation_L[3,i],
#          relative_strength[4,i]/om_r, photon_occupation_L[4,i])




#|c_o^1|^2

In [ ]:
for i in range(len(relative_strength)):
    print(lambda_list[i], photon_occupation_zo[i], photon_occupation_fo[i], photon_occupation_so[i], 
          photon_occupation[i])

In [ ]:
# loop over the different bond-lengths, create different instances
# of MgH+ molecule and compute and store various quantities
n_state = 0
l_ctr = 0
mol = psi4.geometry(molstr)
e, wfn = psi4.energy("scf/cc-pVDZ", return_wfn=True, molecule=mol)
for l in lambda_list:
    lam = np.array([0, 0, l])
    #mol = psi4.geometry(molstr)
    #e, wfn = psi4.energy("scf/cc-pVDZ", return_wfn=True, molecule=mol)
    
    g_ctr = 0
    for g in gamma_list:
        om = om_r - g/2 * 1j
        cqed_dict = cs_cqed_cis(lam, om, molstr, options_dict)
        g_c = np.sqrt(om/2) * l * max_tdm
        relative_strength[g_ctr, l_ctr] = np.real( np.sqrt( np.conj(g_c) * g_c ))
        CL = cqed_dict['ORIGINAL L VECTROS']
        po_l = 0.
        for i in range(0, len(CL[:,0])):
            if i%2==1:
                po_l += np.real(np.conj(CL[i,n_state]) * CL[i,n_state])
        photon_occupation_L[g_ctr, l_ctr] = po_l
        g_ctr += 1
    l_ctr += 1